# Сравнение пользователей музыкального сервиса из двух городов 

# Содержание

Часть 1. Описание проекта.<br>

* 1. Описание исследования
* 2. Цель исследования

Часть 2. Предобработка данных.<br>

* 1. Изучение данных
* 2. Исправление заголовков
* 3. Обработка пропусков
* 4. Обработка дубликатов

Часть 3. Проверка гипотез.<br>

* 1. Сравнение поведения пользователей двух столиц
* 2. Музыка в начале и в конце недели
* 3. Жанровые предпочтения в Москве и Петербурге

Часть 4. Выводы.<br>

* 1. Общий вывод по работе

## Часть. Описание проекта
### Описание исследования
Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На основе предложенных данных проводится сравнение поведения пользователей двух столиц.

О качестве данных ничего не известно, поэтому перед проверкой гипотез необходимо провести обзор данных:
1) Проверить данные на ошибки и оценить их влияние на исследование.<br>
2) На этапе предобработки найти способы исправить самые критичные ошибки данных.

### Цель исследования:
Проверить три гипотезы:
1. Активность пользователей зависит от дня недели, причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

## Часть. Предобработка данных
### Изучение данных

In [1]:
# Импорт библиотеки pandas
import pandas as pd 

In [2]:
# Чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/yandex_music_project.csv')

In [3]:
# Получение первых 10 строк таблицы df
display(df.head(10))

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
# Получение общей информации о данных в таблице df
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


В таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. строчные буквы сочетаются с прописными
2. встречаются пробелы
3. встречается "верблюжий регистр"

Количество значений в столбцах различается, значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

### Исправление заголовков

In [5]:
# Перечень названий столбцов таблицы df
df.columns 

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [6]:
# Переименование столбцов
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'}) 

Проверка результата.

In [7]:
# Проверка результата - перечень названий столбцов
df.columns 

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Обработка пропусков
Для начала стоит рассчитать количество пропущенных значений в таблице.

In [8]:
# Подсчёт пропусков
df.isna().sum() 

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование, так в `track` и `artist` пропуски не важны для вашей работы, достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге, для этого придётся сделать следующее:
* заполнить и эти пропуски явными обозначениями
* оценить, насколько они повредят расчётам. 

Для замены значений в столбцах `track`, `artist` и `genre` создается список `columns_to_replace`, в который входят названия этих столбцов, все его элементы перебераются циклом `for` и для каждого столбца выполняется замена пропущенных значений на строку `'unknown'`.

In [9]:
# Названия столбцов, для замены значений
columns_to_replace = ['track', 'artist', 'genre'] 
# Перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

Проверка результата.

In [10]:
# Подсчёт пропусков
df.isna().sum() 

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Обработка дубликатов
Стоит рассчитать количество дубликатов в таблице.

In [11]:
# Подсчёт явных дубликатов
df.duplicated().sum() 

3826

In [12]:
# Удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True) 

Проверка результата.

In [13]:
# Проверка на отсутствие дубликатов
df.duplicated().sum()

0

Также стоит избавиться от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования. Для этого на экран выводится список уникальных названий жанров, отсортированный в алфавитном порядке.

In [14]:
# Просмотр уникальных названий жанров
df['genre'].sort_values().unique() 

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

При просмотре списка найдены неявные дубликаты названия `hiphop`, возможно это названия с ошибками или альтернативные названия жанра:
* *hip*
* *hop*
* *hip-hop*

Чтобы очистить от них таблицу, можно написать функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов
* `correct_genre` — строка с правильным значением

Функция должна исправить колонку `genre` в таблице `df`: заменить каждое значение из списка `wrong_genres` на значение из `correct_genre`.

In [15]:
# Функция для замены неявных дубликатов
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre)

Далее идет применение функции `replace_wrong_genres()` для устранения неявных дубликатов: вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`.

In [16]:
# Устранение неявных дубликатов
duplicates = ['hip', 'hop', 'hip-hop']
correct = 'hiphop'

replace_wrong_genres(duplicates, correct)

Вывод отсортированного список уникальных значений столбца `genre`, для проверки результата.

In [17]:
# Проверка на неявные дубликаты
df['genre'].sort_values().unique() 

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков
- пропущенные значения
- дубликаты — явные и неявные.

В ходе предобработки исправлены заголовки, чтобы упростить работу с таблицей, убраны дубликаты, чтобы исследование стало более точным. Пропущенные значения зкменены на `'unknown'`, также предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Это предположение можно проверить по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого надо:

* Разделить пользователей Москвы и Санкт-Петербурга
* Сравнить, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

Для оценки активности пользователей в каждом городе, стоит сгруппировать данные по городу и посчитать прослушивания в каждой группе.

In [18]:
# Подсчёт прослушиваний в каждом городе
df.groupby('city')['user_id'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь данные группируются по дню недели и рассчитываются прослушивания в понедельник, среду и пятницу, стоит учесть, что в данных есть информация только о прослушиваниях только за эти дни.

In [19]:
# Подсчёт прослушиваний в каждый из трёх дней
df.groupby('day')['user_id'].count() 

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Теперь можно написать функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели
* название города

В функции в переменную сохраняются строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`
  * в колонке `city` равно параметру `city`

Для этого применяется последовательная фильтрация с логической индексацией.

Затем рассчитываются значения в столбце `user_id` получившейся таблицы. Результат сохраняется в новую переменную, которая возвращается из функции.

In [20]:
# Создание функции number_tracks()
def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Функция `number_tracks()` вызывается шесть раз, при этом менется значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [21]:
# Количество прослушиваний в Москве по понедельникам
print(number_tracks('Monday', 'Moscow'))

15740


In [22]:
# Количество прослушиваний в Санкт-Петербурге по понедельникам
print(number_tracks('Monday', 'Saint-Petersburg'))

5614


In [23]:
# Количество прослушиваний в Москве по средам
print(number_tracks('Wednesday', 'Moscow'))

11056


In [24]:
# Количество прослушиваний в Санкт-Петербурге по средам
print(number_tracks('Wednesday', 'Saint-Petersburg'))

7003


In [25]:
# Количество прослушиваний в Москве по пятницам
print(number_tracks('Friday', 'Moscow'))

15945


In [26]:
# Количество прослушиваний в Санкт-Петербурге по пятницам
print(number_tracks('Friday', 'Saint-Petersburg'))

5895


С помощью конструктора `pd.DataFrame` создается таблица, для хранения результатов, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`
* данные — результаты, которые вы получили с помощью `number_tracks`

In [27]:
# Таблица с результатами
number_tracks = [['Moscow', 15740, 11056, 15945],
                ['Saint-Petersburg', 5614, 7003, 5895]]
columns = ['city', 'monday', 'wednesday', 'friday']
pd.DataFrame(data = number_tracks,columns = columns)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Стоит создать две переменные, для хранения таблиц с данными:
* по Москве — в `moscow_general`
* по Санкт-Петербургу — в `spb_general`

In [28]:
# Получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

In [29]:
# Получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

Функция `genre_weekday()` возвращает информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [30]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    return(genre_df_sorted.head(10))

Дялее следует применение функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00).

In [31]:
# Вызов функции для утра понедельника в Москве
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [32]:
# Вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [33]:
# Вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [34]:
# Вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Для создания таблицы `moscow_genres`, таблица `moscow_general` группируется по жанру и рассчитываются прослушивания треков каждого жанра методом `count()`, затем результат сортируется в порядке убывания и сохраняется.

In [35]:
# Создание таблицы moscow_genres
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending = False)

In [36]:
# Просмотр первых 10 строк moscow_genres
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Аналогично создается таблица `spb_genres`.

In [37]:
# Создание таблицы spb_genres
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending = False)

In [38]:
# Просмотр первых 10 строк spb_genres
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Проверены три гипотезы и получены следующие результаты:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.